<a href="https://colab.research.google.com/github/dimitrod/ehu_nlp_dimathina/blob/Evaluation/Evaluation_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mandarjoshi90/triviaqa.git

Cloning into 'triviaqa'...
remote: Enumerating objects: 70, done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 70 (from 1)
Receiving objects: 100% (70/70), 20.60 KiB | 781.00 KiB/s, done.
Resolving deltas: 100% (28/28), done.


In [2]:
%cd triviaqa

/content/triviaqa


In [3]:
!pip install -r requirements.txt

In [4]:
import subprocess
import os
import json
from tqdm import tqdm
import importlib
import argparse

In [5]:
triviaqa_root = "/content/triviaqa"

In [6]:
env = os.environ.copy()
env["PYTHONPATH"] = triviaqa_root

In [7]:
%cd ..

/content


In [8]:
!pip install datasets transformers huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [24]:
def evaluate_triviaqa(dataset_file, prediction_file):
  script_path = "/content/triviaqa/evaluation/triviaqa_evaluation.py"
  result = subprocess.run([
    "python", script_path,
    "--dataset_file", dataset_file,
    "--prediction_file", prediction_file],
    env=env,
    capture_output=True,
    text=True,
  )
  print(result.stdout)

In [25]:
def get_answers(directory, questions, model):
    answers = {}

    for i in tqdm(range(len(questions)), desc="Retrieving Answers"):
        question = questions[i]
        answers[question[0]] = model.invoke(question[1])
    write_data(directory + "/predictions.json", answers)

In [22]:
def load_questions(path, split):
    create_split(path, split)

    with open(f"{path}/evaluation_dataset.json", "r", encoding="utf-8") as f:
        data = json.loads(f.read())

    questions = []

    for i in tqdm(range(len(data["Data"])),desc="Loading Questions"):
        question = data["Data"][i]
        questions.append([question["QuestionId"], question["Question"]])
    return questions

In [26]:
def create_split(path, split):
  with open(f"{path}/original_dataset.json", "r", encoding="utf-8") as f:
    data = json.loads(f.read())

  data["Data"] = data["Data"][:split]

  write_data(f"{path}/evaluation_dataset.json", data)

In [12]:
def load_model(path, model_name):
  module = importlib.import_module(path)
  model = getattr(module, model_name)
  print("Loading model...")
  return model()

In [27]:
def write_data(path, data):
  with open(path, "w", encoding="utf-8") as f:
    f.write(json.dumps(data))

In [13]:
def get_args():
    parser = argparse.ArgumentParser(
        description='Evaluation Chain for TriviaQA {}')
    parser.add_argument('--dataset_file', help='Dataset file')
    parser.add_argument('--model_file', help='Model file')
    args = parser.parse_args()
    return args

In [30]:
if __name__ == "__main__":
    #args = get_args()
    directory = "/content/evaluation_data"
    split = 10
    model_path = "model.tiny_llama"
    model_name = "tinyllama"


    questions = load_questions(directory, split)
    model = load_model(model_path, model_name)
    print("Model initiated")
    print("Starting QA...")
    answers = get_answers(directory, questions, model)
    print("QA complete")
    print("Test results:")
    evaluate_triviaqa(directory + "/evaluation_dataset.json", directory + "/predictions.json")


Loading Questions: 100%|██████████| 10/10 [00:00<00:00, 10743.61it/s]


Loading model...
Model initiated
Starting QA...


Retrieving Answers: 100%|██████████| 10/10 [10:27<00:00, 62.78s/it]

QA complete
Test results:
em=0: Dame Judi Dench was born in London, England. ['york yorkshire', 'eoferwic', 'park grove primary school', 'park grove school', 'weather in york', 'park grove 1895', 'eoforwic', 'county borough of york', 'york uk', 'un locode gbyrk', 'city of york', 'york england', 'york ua', 'york ham', 'york', 'yorkish', 'yoisk', 'york north yorkshire']
em=0: Angola achieved independence from Portugal on January 10, 1975. ['portugul', 'portugallu', 'portugalska', 'pòtigal', 'portugaul', 'portugalujo', 'portuguese republic', 'iso 3166 1 pt', 'republic of portugal', 'portugalsko', 'portugual', 'bồ đào nha', 'portugall', 'portûnga', 'bo dao nha', 'phortaingeil', 'portugale', 'portugal', 'portugál', 'portugalėjė', 'portiwgal', 'phu to ga', 'portugalija', 'portugalio', 'portogallo', 'phû tô gâ', 'portegal', 'república portuguesa', 'portugāle', 'phortaingéil', 'yn phortiugal', 'portogało', 'portuga', 'portugaleje', 'portekiz', 'o papagaio', 'portunga', 'potigal', 'portekîz', '